In [1]:
import pandas as pd
import math


In [3]:
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 100)

In [5]:
# install the ucimlrepo
%pip install ucimlrepo

Note: you may need to restart the kernel to use updated packages.


In [7]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
diabetes_data = fetch_ucirepo(id=296) 
  
# features and ids as pandas DataFrames
data_ids = diabetes_data.data.ids
data_features = diabetes_data.data.features
data_target = diabetes_data.data.targets

C:\Users\kehin\anaconda3\Lib\site-packages\ucimlrepo\fetch.py:97: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_url)


In [9]:
import pandas as pd

# Load the dataset
file_path = r'C:\Users\kehin\Downloads\diabetes+130-us+hospitals+for+years+1999-2008\diabetic_data.csv'
df = pd.read_csv(file_path)

# Preview the first few rows
print(df.head())

   encounter_id  patient_nbr             race  gender      age weight  admission_type_id  \
0       2278392      8222157        Caucasian  Female   [0-10)      ?                  6   
1        149190     55629189        Caucasian  Female  [10-20)      ?                  1   
2         64410     86047875  AfricanAmerican  Female  [20-30)      ?                  1   
3        500364     82442376        Caucasian    Male  [30-40)      ?                  1   
4         16680     42519267        Caucasian    Male  [40-50)      ?                  1   

   discharge_disposition_id  admission_source_id  time_in_hospital  ... citoglipton insulin  \
0                        25                    1                 1  ...          No      No   
1                         1                    7                 3  ...          No      Up   
2                         1                    7                 2  ...          No      No   
3                         1                    7                 2 

In [11]:
# add a dummy column to merge
data_ids.loc[:, 'merge_temp'] = data_ids.index
data_features.loc[:, 'merge_temp'] = data_features.index
data_target.loc[:, 'merge_temp'] = data_target.index

# merge into one DataFrame
data_temp = data_ids.merge(data_features, on='merge_temp')

data_raw = data_temp.merge(data_target, on='merge_temp')

del data_temp

data_raw = data_raw.drop(columns = 'merge_temp')

print(data_raw.columns, '\n')

print(data_raw.shape)

C:\Users\kehin\AppData\Local\Temp\ipykernel_4104\2302750055.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_ids.loc[:, 'merge_temp'] = data_ids.index
C:\Users\kehin\AppData\Local\Temp\ipykernel_4104\2302750055.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_features.loc[:, 'merge_temp'] = data_features.index
C:\Users\kehin\AppData\Local\Temp\ipykernel_4104\2302750055.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital', 'payer_code',
       'medical_specialty', 'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1', 'diag_2', 'diag_3',
       'number_diagnoses', 'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide',
       'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide',
       'examide', 'citoglipton', 'insulin', 'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone', 'change',
       'diabetesMed', 'readmitted'],
      dtype='object') 

(101766, 50)


In [13]:
# Check available columns
print(df.columns)

# Define your target column (for example, predicting readmission)
target_col = 'readmitted'  # or 'readmit_binary' if you already created it

# Create features (X) and target (y)
X = df.drop(columns=[target_col])
y = df[target_col]

# If you're going to merge again after preprocessing, use df.merge() carefully.

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital', 'payer_code',
       'medical_specialty', 'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1', 'diag_2', 'diag_3',
       'number_diagnoses', 'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide',
       'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide',
       'examide', 'citoglipton', 'insulin', 'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone', 'change',
       'diabetesMed', 'readmitted'],
      dtype='object')


In [15]:
# remove missing/unnecesary features
dropped_columns = ['encounter_id', 'weight', 'payer_code', 'medical_specialty']

data_raw = data_raw.drop(columns = dropped_columns)

print(data_raw.shape)

(101766, 46)


In [17]:
import pandas as pd

# Load the CSV file into a DataFrame named data_raw
file_path = r'C:\Users\kehin\Downloads\diabetes+130-us+hospitals+for+years+1999-2008\diabetic_data.csv'
data_raw = pd.read_csv(file_path)

# Now drop the unwanted columns
dropped_columns = ['encounter_id', 'weight', 'payer_code', 'medical_specialty']
data_raw = data_raw.drop(columns=dropped_columns)

print(data_raw.shape)

(101766, 46)


In [19]:
# Check unique values in 'race' before cleaning
print(df['race'].unique())

['Caucasian' 'AfricanAmerican' '?' 'Other' 'Asian' 'Hispanic']


In [21]:
import pandas as pd
import numpy as np

# Load the data
df = pd.read_csv(file_path)

# Drop unneeded columns
columns_to_drop = ['encounter_id', 'weight', 'payer_code', 'medical_specialty']
df = df.drop(columns=columns_to_drop)

# Remove rows with missing race
df = df[df['race'] != '?']  # Fix is here!

# Replace remaining NaNs with -1
df = df.replace(np.nan, -1)

# Replace specific columns' missing values with 'N/A'
cols_to_replace = ['diag_1', 'diag_2', 'diag_3', 'A1Cresult', 'max_glu_serum']
df[cols_to_replace] = df[cols_to_replace].replace(-1, 'N/A')

# Check shape
print(df.shape)  # This should now print (99493, 46)


(99493, 46)


In [23]:
expired_or_hospice_ids = [
    11,  # Expired
    13,  # Hospice/home
    14,  # Hospice/medical facility
    19,  # Expired at home (Medicaid only, hospice)
    20,  # Expired in a medical facility (Medicaid only, hospice)
    21   # Expired, place unknown (Medicaid only, hospice)
]

df = df[~df['discharge_disposition_id'].isin(expired_or_hospice_ids)]

print(df.shape)

(97109, 46)


In [25]:
# write cleaned (no missing values) dataset to file
df.to_csv('./data/dataset_clean.csv', index=False)


In [27]:
import os

# Create the 'data' folder if it doesn't exist
os.makedirs('data', exist_ok=True)

# Now save the dataframe
df.to_csv('./data/dataset_clean.csv', index=False)

In [29]:
pip install scikit-learn pandas numpy

Note: you may need to restart the kernel to use updated packages.


In [115]:
import pandas as pd

# Load the cleaned data
df = pd.read_csv('./data/dataset_clean.csv')

# Load cleaned data
df = pd.read_csv('./data/dataset_clean.csv')

# Create binary target
df['readmit_binary'] = df['readmitted'].apply(lambda x: 1 if x == '<30' else 0)


In [117]:
  # Define features and target
X = df.drop(columns=['readmitted', 'readmit_binary', 'patient_nbr'])
y = df['readmit_binary']


In [119]:
 from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [120]:
 model = LogisticRegression(max_iter=1000, class_weight='balanced')
model.fit(X_train_scaled, y_train)


LogisticRegression(class_weight='balanced', max_iter=1000)

In [99]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [122]:
 y_pred = model.predict(X_test_scaled)

from sklearn.metrics import confusion_matrix, classification_report

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[1363 1040 1004]
 [2942 4324 3398]
 [3383 3892 9184]]
              precision    recall  f1-score   support

         <30       0.18      0.40      0.25      3407
         >30       0.47      0.41      0.43     10664
          NO       0.68      0.56      0.61     16459

    accuracy                           0.49     30530
   macro avg       0.44      0.45      0.43     30530
weighted avg       0.55      0.49      0.51     30530

